# 🏭 Programação Linear — Mix Ótimo de Produção
**Pós-Graduação BI & Analytics · Bloco 8 — Demonstração**

---

### 📡 Sinal de Negócio
> Relatório mensal mostra que a **margem ficou 12% abaixo do orçado** mesmo com vendas no volume planejado.  
> O controller identifica que o mix vendido não foi o ótimo — a força de vendas priorizou volume, não margem.  
> Pergunta: **qual combinação de produtos maximiza a margem dado que temos capacidade limitada?**

---
### O Problema
Uma fábrica produz 3 produtos: **Alpha**, **Beta** e **Gamma**.  
Cada produto consome recursos diferentes (horas-máquina, horas-homem, matéria-prima).  
Queremos maximizar a **margem de contribuição total** respeitando as restrições de capacidade.

In [ ]:
# ── CÓDIGO COMPLETO — sem lacunas (demonstração) ──────────────────
from scipy.optimize import linprog
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({'figure.dpi': 120, 'axes.spines.top': False, 'axes.spines.right': False})
print('✅ Bibliotecas carregadas')

In [ ]:
# ── PARÂMETROS DO PROBLEMA ──────────────────────────────────────────
PRODUTOS = ['Alpha', 'Beta', 'Gamma']

# Margem de contribuição por unidade (R$)
margem = [120, 180, 95]

# Consumo de recursos por unidade
#                 Alpha  Beta  Gamma
h_maquina  = [    2.0,   3.5,   1.5]   # horas-máquina / unidade
h_homem    = [    1.5,   2.0,   2.5]   # horas-homem / unidade
mat_prima  = [    4.0,   3.0,   5.0]   # kg / unidade

# Capacidade disponível por mês
cap_maquina = 420   # horas
cap_homem   = 380   # horas
cap_materia = 600   # kg

# Demanda máxima por produto (contrato de clientes)
dem_max = [80, 60, 100]

# Tabela de parâmetros
param_df = pd.DataFrame({
    'Produto': PRODUTOS,
    'Margem (R$)': margem,
    'h-Máquina/u': h_maquina,
    'h-Homem/u': h_homem,
    'Mat.Prima/u (kg)': mat_prima,
    'Dem.Máx (u)': dem_max
})
print('Parâmetros do problema:')
print(param_df.to_string(index=False))
print(f'\nCapacidades: Máquina={cap_maquina}h | Homem={cap_homem}h | Mat.Prima={cap_materia}kg')

In [ ]:
# ── FORMULAÇÃO E SOLUÇÃO ───────────────────────────────────────────
# linprog minimiza → negamos a função objetivo
c     = [-m for m in margem]

# Restrições: Ax ≤ b
A_ub  = [
    h_maquina,   # horas-máquina ≤ cap_maquina
    h_homem,     # horas-homem   ≤ cap_homem
    mat_prima,   # matéria-prima ≤ cap_materia
    [1, 0, 0],   # Alpha ≤ dem_max[0]
    [0, 1, 0],   # Beta  ≤ dem_max[1]
    [0, 0, 1],   # Gamma ≤ dem_max[2]
]
b_ub  = [cap_maquina, cap_homem, cap_materia] + dem_max
bounds = [(0, None)] * 3   # produção não-negativa

# Resolver com método HiGHS
resultado = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

prod_otima   = resultado.x
margem_total = -resultado.fun

# ── Resultados ─────────────────────────────────────────────────────
print('='*55)
print('  SOLUÇÃO ÓTIMA — MIX DE PRODUÇÃO')
print('='*55)
for i, p in enumerate(PRODUTOS):
    print(f'  {p:<8}: {prod_otima[i]:>6.1f} unidades  '
          f'→ Margem: R$ {prod_otima[i]*margem[i]:>8,.0f}')
print(f'  {"":─<44}')
print(f'  {"TOTAL":<8}  {sum(prod_otima):>6.0f} unidades  '
      f'→ Margem: R$ {margem_total:>8,.0f}')

# Utilização de recursos
print('\n  Utilização de Recursos:')
recursos = [('Horas-Máquina', h_maquina, cap_maquina),
            ('Horas-Homem',   h_homem,   cap_homem),
            ('Matéria-Prima', mat_prima, cap_materia)]

for nome, coefs, cap in recursos:
    uso   = sum(c*q for c,q in zip(coefs, prod_otima))
    folga = cap - uso
    gargalo = ' ← GARGALO' if folga < 0.5 else ''
    print(f'  {nome:<15}: {uso:>6.1f}/{cap} ({uso/cap*100:.0f}%) '
          f'folga={folga:.1f}{gargalo}')

print(f'\n  Status solver: {resultado.message}')

In [ ]:
# ── VISUALIZAÇÃO ───────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
cores = ['#c84b2f', '#2f6ec8', '#2a8c5a']

# Mix ótimo
bars = axes[0].bar(PRODUTOS, prod_otima, color=cores)
axes[0].bar(PRODUTOS, [d - p for d, p in zip(dem_max, prod_otima)],
            bottom=prod_otima, color=[c + '33' for c in ['#c84b2f','#2f6ec8','#2a8c5a']],
            label='Demanda não atendida')
axes[0].set(title='Mix Ótimo (unidades)', ylabel='Unidades/mês')
for bar, v in zip(bars, prod_otima):
    axes[0].text(bar.get_x() + bar.get_width()/2, v + 0.5,
                 f'{v:.0f}', ha='center', fontsize=11, fontweight='bold')
axes[0].legend(fontsize=8)

# Contribuição à margem
margens_p = [p*m for p, m in zip(prod_otima, margem)]
axes[1].bar(PRODUTOS, margens_p, color=cores)
axes[1].set(title='Contribuição à Margem (R$)', ylabel='R$')
for i, v in enumerate(margens_p):
    axes[1].text(i, v + 100, f'R${v:,.0f}', ha='center', fontsize=9)

# Utilização de capacidade
nomes_r = ['Máquina', 'Homem', 'Mat.Prima']
utils   = []
for coefs, cap in zip([h_maquina, h_homem, mat_prima],
                      [cap_maquina, cap_homem, cap_materia]):
    uso = sum(c*q for c, q in zip(coefs, prod_otima))
    utils.append(uso/cap*100)

barsh = axes[2].barh(nomes_r, utils,
                     color=['#c84b2f' if u > 98 else '#94a3b8' for u in utils])
axes[2].axvline(100, color='#1a1a1a', lw=1.2, linestyle='--')
axes[2].set(title='Utilização de Capacidade (%)', xlim=(0, 115))
for bar, u in zip(barsh, utils):
    axes[2].text(u + 1, bar.get_y() + bar.get_height()/2,
                 f'{u:.0f}%', va='center', fontsize=10,
                 color='#c84b2f' if u > 98 else '#1a1a1a',
                 fontweight='bold' if u > 98 else 'normal')

plt.suptitle(f'Mix Ótimo → Margem Total: R$ {margem_total:,.0f}/mês',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()
print('\n💡 O gargalo define o teto da operação.')
print('   Para crescer margem: investir nesse recurso, não nos outros.')

In [ ]:
# ── ANÁLISE DE SENSIBILIDADE: e se relaxarmos o gargalo? ──────────
print('Análise de Sensibilidade — Impacto de Ampliar Cada Recurso\n')
print(f"{'Recurso':<18} {'Cap. Atual':>12} {'Cap. +10%':>12} {'Δ Margem':>12}")
print('-' * 58)

caps_originais = [cap_maquina, cap_homem, cap_materia]
nomes_rec = ['Horas-Máquina', 'Horas-Homem', 'Mat.Prima']

for idx_r, (nome, cap_orig) in enumerate(zip(nomes_rec, caps_originais)):
    caps_novo = caps_originais.copy()
    caps_novo[idx_r] = cap_orig * 1.10  # +10%
    b_novo = caps_novo + dem_max
    res_novo = linprog(c, A_ub=A_ub, b_ub=b_novo, bounds=bounds, method='highs')
    delta = -res_novo.fun - margem_total
    print(f'  {nome:<16} {cap_orig:>12,.0f} {cap_orig*1.1:>12,.0f} '
          f'R$ {delta:>+9,.0f}')

print()
print('💡 O recurso com maior Δ Margem por unidade adicional')
print('   é onde o investimento em capacidade tem maior ROI.')